In [0]:
%sql
use catalog data_modeling;

In [0]:
%sql
create or replace table silver.customer as
select 
cast(COALESCE(UserId, 0) as long) as UserId,
trim(COALESCE(PhoneNumber, '')) as PhoneNumber,
trim(COALESCE(CustName, '')) as CustName,
trim(COALESCE(Gender, '')) as Gender,
cast(COALESCE(Age, 0) as int) as Age,
trim(COALESCE(MaritalStatus, 0)) as MaritalStatus,
trim(COALESCE(State, '')) as State,
trim(COALESCE(Zone, '')) as Zone,
trim(COALESCE(Occupation, '')) as Occupation
-- ,
-- CreatedDate
from bronze.raw_sales_data
qualify row_number() over(partition by UserId order by PhoneNumber) = 1;

-- select * from silver.customer

In [0]:
%sql
create or replace table silver.product as
select *
from (
select
  cast(COALESCE(ProductId, 0) as long) as ProductId,
  trim(COALESCE(ProductCode, '')) as ProductCode,
  trim(COALESCE(ProductDesc, '')) as ProductDesc,
  trim(COALESCE(ProductCategory, '')) as ProductCategory,
  trim(COALESCE(ProductSubCategory, '')) as ProductSubCategory,
  avg(ProductCost) as ProductCost
  -- ,
  -- CreatedDate
from bronze.raw_sales_data
group by cast(COALESCE(ProductId, 0) as long),
trim(COALESCE(ProductCode, '')), 
trim(COALESCE(ProductDesc, '')),
trim(COALESCE(ProductCategory, '')),
trim(COALESCE(ProductSubCategory, '')),
CreatedDate
) as df;

-- select * from silver.product

In [0]:
%sql
create or replace table silver.loc as
select
cast(COALESCE(StoreID, 0) as int) as StoreID,
trim(COALESCE(State, '')) as State,
trim(COALESCE(Zone, '')) as Zone,
trim(COALESCE(Region, '')) as Region,
trim(COALESCE(Country, '')) as Country
-- ,
-- CreatedDate
from bronze.raw_sales_data
qualify row_number() over(partition by StoreID order by OrderTms desc) = 1;

-- select * from silver.loc ;

In [0]:
%sql
create or replace table silver.sales as
select 
  cast(RunID as string) as RunID,
  cast(trim(coalesce(TransactionId, '0')) as long) as TransactionId,
  cast(trim(coalesce(UserId, '0')) as long) as UserId,
  cast(trim(coalesce(ProductId, '0')) as long) as ProductId,
  cast(trim(coalesce(StoreID, '0')) as long) as StoreID,
  cast(OrderTms as timestamp) as OrderTms,
  date(OrderTms) as OrderDate,
  cast(trim(coalesce(Orders, '0')) as double) as OrderCount,
  cast(trim(coalesce(Amount, '0')) as double) as TotalAmount,
  cast(CreatedDate as timestamp)  as CreatedDate
from bronze.raw_sales_data;

-- select * from silver.sales